In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2

In [26]:
N = 600
dx = 12.5e-6
L = N * dx

lam = 532e-9
k = 2 * np.pi / lam
z = -120e-3

x = np.linspace(-L/2, L/2, N)
y = x
X, Y = np.meshgrid(x, y)

In [28]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import cv2

um = 1e-6
mm = 1e-3

def ASM(image, lam, dx, z):
    k = 2 * np.pi / lam
    N = image.shape[0]
    df = 1 / N / dx
    
    fx = np.arange(-N//2, N//2) * df
    fy = fx

    Fx, Fy = np.meshgrid(fx, fy)
    
    kz_squared = k**2 - (2 * np.pi * Fx)**2 - (2 * np.pi * Fy)**2
    kz_squared[kz_squared<0] = 0
    kz = np.sqrt(kz_squared)
    Hz = np.exp(1j * kz * z)
    
    fimage = np.fft.fftshift(np.fft.fft2(image))
    fimage = fimage * Hz
    pimage = np.fft.ifft2(np.fft.ifftshift(fimage))
    
    return pimage

In [29]:
image = Image.open('target.png').convert('L')
image = np.array(image)
image = cv2.resize(image, dsize=(N,N))
# image = 255 - image

In [34]:
z = -80e-3
image = ASM(image, lam, 12.5*um, z)
image = np.abs(image)**2
image = image / np.max(image) * 255
Image.fromarray(image.astype('uint8')).save('test.png')

In [15]:
def fresnel(image, lam, z):
    coeff = np.exp(1j * k * z) / (1j * lam * z)
    kernel = coeff * np.exp(1j * k / 2 / z * (X**2 + Y**2))
    transfer = np.fft.fftshift(np.fft.fft2(kernel))

    f_image = np.fft.fftshift(np.fft.fft2(image))
    f_image = f_image * transfer
    image = np.fft.ifft2(np.fft.ifftshift(f_image))
    image = np.fft.fftshift(image) # Don't know why do this but this should be exist
    return image

In [27]:
image = Image.open('target.png').convert('L')
image = np.array(image)
image = cv2.resize(image, dsize=(N,N))
# image = 255 - image

coeff = np.exp(1j * k * z) / (1j * lam * z)
kernel = coeff * np.exp(1j * k / 2 / z * (X**2 + Y**2))
transfer = np.fft.fftshift(np.fft.fft2(kernel))

f_image = np.fft.fftshift(np.fft.fft2(image))
f_image = f_image * transfer
image = np.fft.ifft2(np.fft.ifftshift(f_image))
image = np.fft.fftshift(image) # Don't know why do this but this should be exist

phase = np.angle(image)
phase = (phase - np.min(phase)) / (np.max(phase) - np.min(phase))
phase = phase * 255
Image.fromarray(phase.astype('uint8')).save('test.png')

In [6]:
from lucam import Lucam

# ‼️ 실제 카메라 촬영 로직
camera = Lucam()
capture = camera.TakeSnapshot()
capture = cv2.resize(capture, dsize=(N, N))
cv2.imwrite('test.png', capture)

True